# C-SWM 

In [1]:
#Libs
using HDF5
using Knet
using Statistics: mean,std
using Random
using LinearAlgebra
import Base: iterate, length


#Datatype
atype=KnetArray{Float32}

#Includes
include("utils.jl")
include("layernorm.jl")
include("encodercnn.jl")
include("encodermlp.jl")
include("gnn.jl")

#Params
SAVE_FOLDER = "./checkpoints"
NUM_STEPS = 1
TRAIN_DATASET_PATH = "/home/cagan/dev/datasets/C-SWM/shapes_train.h5"
BATCH_SIZE = 100
SEED = 0
NUM_OBJECTS = 5

5

In [2]:
dtrn = buildDataset(TRAIN_DATASET_PATH, true, BATCH_SIZE);

Dataset loaded. Building dataset indexing...
Done.


In [30]:
obs,action,next_obs = first(dtrn)
println(size(obs))
println(size(action))
println(size(next_obs))

(50, 50, 3, 100)
(100,)
(50, 50, 3, 100)


In [3]:
#Params
input_ch = 3
hidden_dim = 512
num_objects = 5
embedding_dim = 2
action_dim = 4
sigma = 0.5
hinge = 1.0

1.0

In [4]:
function energy(transition_model, state, action, next_state, trans)
    
    norm = 0.5 / (sigma^2)
    
    if trans
        
        diff = state - next_state
        
    else
        
        pred_trans = transition_model(state,action)
        diff = state + pred_trans - next_state
        
    end
    
    #Diff has shape => (2,5,100)
    #Result has shape => (100)
    return mean(norm*sum((diff.^2), dims=1),dims=1)
    
    
end

energy (generic function with 1 method)

In [5]:
mutable struct ContrastiveSWM
    
    obj_extractor::EncoderCNNSmall
    obj_encoder::EncoderMLP
    gnn::TransitionGNN
    
end

In [6]:
function initContrastiveSWMSmall()

    obj_extractor = initEncoderCNNSmall(input_ch, hidden_dim ÷ 16, num_objects, sigm, relu)
    obj_encoder = initEncoderMLP(25, hidden_dim, embedding_dim, num_objects, relu)
    gnn = initTransitionGNN(embedding_dim, hidden_dim, action_dim, num_objects, false, false, relu)
    
    return ContrastiveSWM(obj_extractor, obj_encoder, gnn)
    
end

initContrastiveSWMSmall (generic function with 1 method)

In [7]:
# Forward propagation with transition
function (m::ContrastiveSWM)(obs,action)
    
    # Extract objects
    objs = m.obj_extractor(obs)
    
    # Obtain embeddings
    state = m.obj_encoder(objs)
    
    #Transition    
    out = m.gnn(state,action)
    
    return out
    
end

In [8]:
# Forward propagation without transition
function (m::ContrastiveSWM)(obs)
    
    # Extract objects
    objs = m.obj_extractor(obs)
    
    # Obtain embeddings
    state = m.obj_encoder(objs)
    
    return out
    
end

In [12]:
# Contrastive loss part
function (m::ContrastiveSWM)(obs,action,next_obs)
    
    # Extract objects
    objs = m.obj_extractor(obs)
    next_objs = m.obj_extractor(next_obs)
    
    # Obtain embeddings
    state = m.obj_encoder(objs)
    next_state = m.obj_encoder(next_objs)
     
    # Sample negative state across episodes at random
    batch_size = size(obs,4)
    perm = rand(1:batch_size)
    neg_state = state[:,:,perm]
    
    # Pos loss
    pos_loss = energy(m.gnn, state, action, next_state,true)
    zero_mat = atype(zeros(size(pos_loss)))
    pos_loss = mean(pos_loss)
    
    # Neg loss
    neg_loss = max.(zero_mat, hinge .- energy(m.gnn, state, action,next_state,false))
    neg_loss = mean(neg_loss)
        
    loss = pos_loss + neg_loss  
    
    return loss
    
end

In [13]:
model = initContrastiveSWMSmall()

ContrastiveSWM(EncoderCNNSmall(Any[P(KnetArray{Float32,4}(10,10,3,32)), P(KnetArray{Float32,4}(1,1,32,5))], Any[Knet.BNMoments(0.1, nothing, nothing, zeros, ones), K32(64)[1.0⋯]], Knet.sigm, NNlib.relu), EncoderMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,25)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(2,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(2))], LayerNorm(P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), 1.0e-6), NNlib.relu), TransitionGNN(EdgeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,4)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(512,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512))], LayerNorm(P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), 1.0e-6), NNlib.relu), NodeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,518)), P(KnetArray{Floa

## Training

In [14]:
NUM_ITER = 10000
VERBOSE =  100

for  (k, (obs, action, next_obs)) in enumerate(dtrn)
    
    #Train by using contrastive loss
    loss = model(obs,action,next_obs)
    
    J = @diff loss 
    for par in params(model)
        g = grad(J, par)
        update!(value(par), g, par.opt)
    end
    
    if k % VERBOSE == 0
       
        println(loss)
        
    end
    
    if k > NUM_ITER
        break
    end
    
        
end

0.0001252732
0.00012831422
8.59368e-5
0.00013691901
0.0001533839
0.0001274361
0.00014394532
8.6288426e-5
0.000115543015
0.0001311372


error in running finalizer: InterruptException()


InterruptException: InterruptException:

## Evaluation